In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building a photo recognition agent: Reasoning Engine setup

This notebook shows how to setup the Reasoning Engine for the photo recognition agent demo

### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Reasoning Engine and LangChain:

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform[langchain,reasoningengine]==1.90.0 google-cloud-discoveryengine==0.11.10

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2026.1.0 requires google-cloud-storage>=3.7.0, but you have google-cloud-storage 2.19.0 which is incompatible.
kfp 2.15.2 requires protobuf<7.0,>=6.31.1, but you have protobuf 4.25.8 which is incompatible.
kfp-pipeline-spec 2.15.2 requires protobuf<7.0,>=6.31.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.39.1 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

<div class="alert alert-block alert-warning">
If prompted, in the <b>Kernel Restarting</b> dialog, click <b>Ok</b>.
</div>

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

Before running this cell to initialize the Vertex AI SDK, update the configuration values for the <b>PROJECT_ID, LOCATION, and STAGING_BUCKET.</b>

In [1]:
PROJECT_ID = "qwiklabs-gcp-02-bb0060d3fbaa"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://qwiklabs-gcp-02-bb0060d3fbaa-lab-bucket"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Wikipedia Tool
In this section, you'll define multiple Python functions that access the Wikipedia API. Later we'll use these Python functions as Tools in our Reasoning Engine agent.

### Define functions for accessing the Wikipedia API

In [2]:
import requests

# get full text of the wiki page
def get_wiki_full_text(wiki_title):
    # get the page
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles={wiki_title}&explaintext=true&format=json"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
    }
    response = requests.get(url, headers=headers)
    data = response.json()

    # extract plain text
    page_id = next(iter(data["query"]["pages"]))
    plain_text = data["query"]["pages"][page_id]["extract"]
    return plain_text

### Define Tool function for the Wikipedia API

In [3]:
# make a query on Wikipedia for a topic
def query_with_wikipedia(
    query: str = "Fallingwater",
):
    """
    Finds answer for any topic or object from Wikipedia and returns a dictionary containing the answer.

    Args:
        query: the name of object or topic to find.

    Example: {"answer": "Fallingwater is a house designed by the architect Frank Lloyd Wright in 1935."}
    """

    wiki_full_text = get_wiki_full_text(query)  # calls Wikipedia API to get full text
    
    return {"answer": wiki_full_text}

In [4]:
query_with_wikipedia()

{'answer': 'Fallingwater is a house museum in Stewart Township in the Laurel Highlands of southwestern Pennsylvania, United States. Designed by the architect Frank Lloyd Wright, it is built partly over a waterfall on the Bear Run stream. The three-story residence was developed as a weekend retreat for Liliane and Edgar J. Kaufmann Sr., the owner of Kaufmann\'s Department Store in Pittsburgh. The Western Pennsylvania Conservancy (WPC), which has operated Fallingwater as a tourist attraction since 1963, maintains 5,000 acres (2,000 ha) surrounding the house. \nEdgar Kaufmann Sr. had established a summer retreat at Bear Run for his employees by 1916. When employees stopped using the retreat, the Kaufmanns bought the site in July 1933 and hired Wright to design the house in 1934. Several structural issues arose during the house\'s construction, including cracked concrete and sagging terraces. The Kaufmanns began using the house in 1937 and hired Wright to design a guest wing, which was fin

## Vertex AI Search Tool
In this section, it will define a Tool for calling the /ask_gms on the Cloud Run for making a query with the Google Merch Shop dataset on Vertex AI Search. So, before using this Tool, you need to deploy the Run instance with /ag-web/app/deploy.sh .

### Define Tool funtion for Vertex AI Search

The Cloud Run host name can be found on Console > Cloud Run > ag-web > URL at the top

In [5]:
from vertexai.preview import reasoning_engines

GOOGLE_SHOP_VERTEXAI_SEARCH_URL = (
    "https://ag-web-570442271344.us-central1.run.app/ask_gms"  # please change
)

In [6]:
def find_product_from_googleshop(product_name: str, product_description: str):
    """
    Find a product with the product_name and product_description from
    Google Merch Shop and returns a dictionary containing product details.
    """

    params = {"query": product_name + " " + product_description}
    response = requests.get(
        GOOGLE_SHOP_VERTEXAI_SEARCH_URL, params
    )  # calls Vertex AI Search
    item = response.json()
    productDetails = f"""
        {item['title']} is a product sold at Google Merch Shop. The price is {item['price']}. 
        You can buy the product at their web site: {item['link']}.
    """

    return {"productDetails": productDetails}

## Define and deploy the Agent
In this section, it creates an agent with the Wikipedia Tool and Search Tool, and deploy it to the Reasoning Engine runtime.

### Define the agent

In [7]:
model_name = "gemini-2.5-flash"  # please change
agent = reasoning_engines.LangchainAgent(
    model=model_name,
    tools=[query_with_wikipedia, find_product_from_googleshop],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

### Test it locally

In [8]:
# input_text = "Google Dino Pin. A collectible enamel pin featuring the iconic Lonely T-Rex dinosaur from the Google Chrome browser offline error page. Great product."
# input_text = "Google Bike Enamel Pin. This is a collectible lapel pin featuring a colorful bicycle with a basket, likely representing Google's company culture and its promotion of cycling as a sustainable mode of transportation."
# input_text = "Android Pen. This is a blue ballpoint pen featuring the Android logo. It appears to be a promotional or branded item associated with the Android operating system"
# input_text = "Google Chrome Dinosaur Game Pin. where can i buy it? Write the answer in plaintext."
input_text = (
    "Google Bike Enamel Pin. where can i buy it? Write the answer in plaintext."
)
agent.query(input=input_text)

{'input': 'Google Bike Enamel Pin. where can i buy it? Write the answer in plaintext.',
 'output': 'You can buy the Google Bike Enamel Pin for 8.00 USD at the Google Merch Shop website: https://shop.merch.google/product/google-bike-enamel-pin-ggoegcba207399/',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'find_product_from_googleshop',
     'tool_input': {'product_description': 'Google Bike Enamel Pin',
      'product_name': 'Google Bike Enamel Pin'},
     'log': "\nInvoking: `find_product_from_googleshop` with `{'product_description': 'Google Bike Enamel Pin', 'product_name': 'Google Bike Enamel Pin'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'find_product_from_goo

### Deploy the Agent to the Reasoning Engine runtime

In [9]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "requests",
    ],
)

Using bucket qwiklabs-gcp-02-bb0060d3fbaa-lab-bucket
Writing to gs://qwiklabs-gcp-02-bb0060d3fbaa-lab-bucket/reasoning_engine/reasoning_engine.pkl
Writing to gs://qwiklabs-gcp-02-bb0060d3fbaa-lab-bucket/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://qwiklabs-gcp-02-bb0060d3fbaa-lab-bucket/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/570442271344/locations/us-central1/reasoningEngines/2248612329476325376/operations/7056127124066795520
ReasoningEngine created. Resource name: projects/570442271344/locations/us-central1/reasoningEngines/2248612329476325376
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/570442271344/locations/us-central1/reasoningEngines/2248612329476325376')


### Test it

In [10]:
remote_agent.query(input="Where can I buy the Chrome Dino Rope Lanyard?")

{'input': 'Where can I buy the Chrome Dino Rope Lanyard?',
 'intermediate_steps': [[{'kwargs': {'tool_input': {'product_description': 'Chrome Dino Rope Lanyard',
      'product_name': 'Chrome Dino Rope Lanyard'},
     'tool': 'find_product_from_googleshop',
     'type': 'AgentActionMessageLog',
     'log': "\nInvoking: `find_product_from_googleshop` with `{'product_name': 'Chrome Dino Rope Lanyard', 'product_description': 'Chrome Dino Rope Lanyard'}`\n\n\n",
     'message_log': [{'kwargs': {'tool_call_chunks': [{'type': 'tool_call_chunk',
          'index': None,
          'id': 'e09e9855-a749-4eb2-9271-7ddcbce9915e',
          'args': '{"product_name": "Chrome Dino Rope Lanyard", "product_description": "Chrome Dino Rope Lanyard"}',
          'name': 'find_product_from_googleshop'}],
        'type': 'AIMessageChunk',
        'tool_calls': [{'type': 'tool_call',
          'id': 'e09e9855-a749-4eb2-9271-7ddcbce9915e',
          'args': {'product_description': 'Chrome Dino Rope Lanyard',


In [11]:
remote_agent.query(input="What is Fallingwater?")

{'input': 'What is Fallingwater?',
 'output': 'Fallingwater is a house museum in Stewart Township in the Laurel Highlands of southwestern Pennsylvania, United States. It was designed by the architect Frank Lloyd Wright and is built partly over a waterfall on the Bear Run stream. The three-story residence was originally developed as a weekend retreat for Liliane and Edgar J. Kaufmann Sr., the owner of Kaufmann\'s Department Store in Pittsburgh. The Western Pennsylvania Conservancy (WPC) has operated Fallingwater as a tourist attraction since 1963 and maintains 5,000 acres (2,000 ha) surrounding the house.\n\nThe house is known for its organic architecture, blending with its natural surroundings. It features multiple cantilevered outdoor terraces, made of locally-quarried stone, reinforced concrete, steel, and plate glass. The first story contains the main entrance, living room, two outdoor terraces, and the kitchen, while the upper stories house four bedrooms and additional terraces. Wr